<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge

In [2]:
import os
import sys

import numpy as np

from deepcell_tracking.trk_io import load_trks

sys.path.append('..')
import utils

In [3]:
data_dir = 'ctc-data'
gt_seg_dir = os.path.join(data_dir, 'gt-seg')
pred_seg_dir = os.path.join(data_dir, 'pred-seg')

models = [
    'Fluo-N2DL-HeLa'
]


for d in [data_dir, gt_seg_dir, pred_seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [4]:
files = ['../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-01.trk', '../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-02.trk']
X, y, lineages = [], [], []
for f in files:
    data = load_trks(f)
    X.append(data['X'])
    y.append(data['y'])
    lineages.append(data['lineages'][0])

data = {
    'X': np.stack(X, axis=0),
    'y': np.stack(y, axis=0),
    'lineages': lineages
}

Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will test.

In [5]:
for batch_no in range(len(data['lineages'])):
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames), ..., 0]
    y = y[:len(good_frames), ..., 0]

    # Save copes of raw and gt data in a folder for each model
    # 2 copies, one for gt seg and one for pred seg
    for m in models:
        # Save pred seg copy
        utils.save_ctc_raw(os.path.join(pred_seg_dir, m), batch_no + 1, X)
        utils.save_ctc_gt(os.path.join(pred_seg_dir, m), batch_no + 1, y, lineage)

        # Save GT seg copy
        utils.save_ctc_raw(os.path.join(gt_seg_dir, m), batch_no + 1, X)
        utils.save_ctc_gt(os.path.join(gt_seg_dir, m), batch_no + 1, y, lineage)
        utils.save_ctc_res(os.path.join(gt_seg_dir, m), batch_no + 1, y, lineage=None, seg=True)